In [1]:
import pandas as pd
import csv
import numpy as np
import os
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem

In [2]:
#data = pd.read_csv('data/alldata.smi',header=None,sep=' ',usecols=[0,2],names=['Smile','Label'])
data = pd.read_csv('../mmodel13/interpret/fort.5005')

In [3]:
data_check=data.sort_values('score')
data_check.head(1)

,smile,score,core_smile,core_score
122,['C1c2c(cc(OC)cc2)[C@]23CCN(C)[C@@H]1[C@@H]2CC...,0.157,NaN,...


In [4]:
print(data_check.loc[0,'smile'])
print(data_check.loc[0,'smile'][2:-2])

['c1(F)ccc([C@H](N2[C@@H]3Cc4c(cc(O)cc4)[C@@](C)([C@@H]3C)CC2)CC)c(Cl)c1']
c1(F)ccc([C@H](N2[C@@H]3Cc4c(cc(O)cc4)[C@@](C)([C@@H]3C)CC2)CC)c(Cl)c1


In [5]:
for i in range(0,len(data_check)):
    data_check.loc[i,'smile']=data_check.loc[i,'smile'][2:-2]

In [6]:
frame_test = data_check
import xlsxwriter
cols = list(frame_test.columns)
print(cols)
cols.remove('smile')
print(cols)
cols.remove('core_smile')
print(cols)
#del dict
dataTypes = dict(frame_test.dtypes)
print(dataTypes)

['smile', 'score', 'core_smile', 'core_score']
['score', 'core_smile', 'core_score']
['score', 'core_score']
{'smile': dtype('O'), 'score': dtype('float64'), 'core_smile': dtype('O'), 'core_score': dtype('O')}


In [7]:
size=(200,200)
print(size[0])
print(size[0]/6.)
outFile='test10_model13_fold0_split.xlsx'
workbook = xlsxwriter.Workbook(outFile)
worksheet = workbook.add_worksheet()
worksheet.set_column('A:B',size[0]/6.)

200
33.333333333333336


0

In [8]:
cell_format1 = workbook.add_format({'bold':True,'font_color':'red'})
cell_format1.set_align('vcenter')
cell_format2 = workbook.add_format()
cell_format2.set_align('vcenter')
c2 = 2
for x in cols:
    worksheet.write_string(0, c2, x)
    c2 += 1

In [9]:
import os
from rdkit.Chem import Draw
from io import BytesIO
from numpy import math
c = 1
for index,row in frame_test.iterrows():
    #print(row)
    image_data1 = BytesIO()
    image_data2 = BytesIO()
    #image_data3 = BytesIO()
    mol1 = Chem.MolFromSmiles(row['smile'])
    if pd.isnull(row['core_smile']):
        mol2 = Chem.MolFromSmiles('*')
    else:
        print(index)
        mol2 = Chem.MolFromSmiles(row['core_smile'])
    
    
    #img1 = Draw.MolToImage(row['Full'],size=(200,200))
    img1 = Draw.MolToImage(mol1,size=(200,200))
    #img2 = Draw.MolToImage(row['part4'],size=(200,200))
    img2 = Draw.MolToImage(mol2,size=(200,200))
    #img3 = Draw.MolToImage(row['part5'],size=(200,200))
    img1.save(image_data1,format='PNG')
    img2.save(image_data2,format='PNG')
    #img3.save(image_data3,format='PNG')
    
    worksheet.set_row(c,height=size[1])
    worksheet.insert_image(c,0,"f",{'image_data':image_data1,'x_offset': 15, 'y_offset': 50})
    worksheet.insert_image(c,1,"f",{'image_data':image_data2,'x_offset': 15, 'y_offset': 50})
    #worksheet.insert_image(c,2,"f",{'image_data':image_data3,'x_offset': 15, 'y_offset': 50})
    
    c2 = 2
    for x in cols:
        if str(dataTypes[x]) == 'object':
            worksheet.write_string(c, c2, str(row[x])[:32000],cell_format1) # string length is limited in xlsx
        elif ('float' in str(dataTypes[x])) or ('int' in str(dataTypes[x])):
            if (row[x] != np.nan) or (row[x] != np.inf):
                worksheet.write_number(c, c2, row[x],cell_format1)
        elif 'datetime' in str(dataTypes[x]):
                worksheet.write_datetime(c, c2, row[x],cell_format1)
        c2 +=1
    c +=1

workbook.close()
image_data1.close()
image_data2.close()
#image_data3.close()

27
18
112
23
113
25
24
10
92
57
46
87
115
103
20
99
88
38
44
80
19
116
34
79
121
9
109
31
36
60
89
29
65
58
7
56
41
64
68
49
74
94
66
84
16
95
33
63
101
47
119
83
5
120
54


In [10]:
PandasTools.AddMoleculeColumnToFrame(data_check,'core_smile','Molecule')
data_check[["core_smile","Molecule",'core_score']].head(2)

TypeError: No registered converter was able to produce a C++ rvalue of type std::__cxx11::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float